In [1]:
import torch
import torch.optim as optim
from torch.autograd import Variable
import pandas as pd
import numpy as np

data = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
data = data.dropna()

In [2]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X = X.to_numpy()
y = y.to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
#unsqueeze function removes all the tensors whose dimension=1
y_train = torch.from_numpy(y_train).float().unsqueeze(1)
y_test = torch.from_numpy(y_test).float().unsqueeze(1)

In [4]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        ''' Linear1 is the input layer
            Linear2 is the hidden layer 
        '''
        self.layer1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(hidden_size, output_size)
        '''
            We are using binary cross-entropy loss function (BCELoss)
            and therefore require the inputs to be in the range of 0-1
            So, we use sigmoid function to do that.'''
        self.sigmoid = torch.nn.Sigmoid()

        ''' First you enter layer1(input layer) and then your output from layer1 
        passes a ReLu and then enters layer2 (hidden layer); 
        whatever outputs from layer2 goes through a sigmoid. 
        Output of the sigmoid function is the final output'''
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.sigmoid(out)
        return out

In [5]:
class AntColonyOptimization():
    def __init__(self, ant_count, epochs, lr, beta, decay, pheromone_level, size):
        self.ant_count = ant_count
        self.epochs = epochs
        self.lr = lr
        self.beta = beta
        self.decay = decay
        self.pheromone_level = pheromone_level
        self.size = size
        self.pheromone = np.ones((size, size)) #pheromone matrix full of ones
        self.shortest_path = np.zeros((1, size))
        self.shortest_path_length = float("inf") #shortest path has been set to infinity
        
    def optimize(self, model, X, y):
        criterion = torch.nn.BCELoss()
        '''SGD is Stochastic Gradient Descent'''
        optimizer = optim.SGD(model.parameters(), lr=self.lr)

        for epoch in range(self.epochs):
            for ant in range(self.ant_count):
                outputs = model(X)
                #criterion( input , target )
                loss = criterion(outputs, y)
                loss.backward()
                self.update_pheromone(loss)
            self.evaporate()
        return self.shortest_path
        
    def prob(self, pheromone, heuristic):
        '''Assign probability to each path taken'''
        probabilities = self.prob(pheromone, heuristic)
        numerator = np.power(pheromone, self.beta) * np.power(heuristic, self.beta)
        denominator = np.sum(numerator)
        probabilities = numerator / denominator
        return probabilities
        
    def update_pheromone(self, loss):
        fitness = 1/loss
        ''' This function basically helps us choose the next path/update shortest path
            If the fitness is less than the shortest path's length, 
            Then shortest path length = fitness
            Shortest path is the one with the pheromones'''
        if fitness < self.shortest_path_length:
            self.shortest_path_length = fitness
            self.shortest_path = self.pheromone
        for i in range(self.size):
            for j in range(self.size):
                '''this is done to allow some level of uncertainty as we dont know
                how many ants travelled the path and how much pheromones changed'''
                self.pheromone[i, j] *= (1 - self.decay)
                self.pheromone[i, j] += self.pheromone_level/fitness
                
    def evaporate(self):
        '''Apply decay on the pheromones because they evaporate with time'''
        self.pheromone *= self.decay

In [6]:
'''Create an instance of the classes AntColony and ACO'''
model = NeuralNet(13,5,1)
optimizer = AntColonyOptimization(ant_count=10, epochs=100, lr=0.1, beta=2, decay=0.5
                                  , pheromone_level=100, size=5)

'''process'''
inputs = Variable(torch.randn(1, 13))
targets = Variable(torch.randn(1, 1))
loss_fn = torch.nn.MSELoss()
for i in range(51):
    optimizer.optimize(model, X_train, y_train)
    output = model(X_train)
    loss = loss_fn(output, y_train)
    loss.backward()
    loss = torch.tensor(loss) #this is done to get the numerical loss part only
    if i%10 == 0:
        print(f"Epoch {i} ; loss={loss}")

C:\Users\Rhea Pandita\AppData\Local\Temp\ipykernel_3596\2567019472.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = torch.tensor(loss) #this is done to get the numerical loss part only


Epoch 0 ; loss=0.2944999933242798
Epoch 10 ; loss=0.2944999933242798
Epoch 20 ; loss=0.2944999933242798
Epoch 30 ; loss=0.2944999933242798
Epoch 40 ; loss=0.2944999933242798
Epoch 50 ; loss=0.2944999933242798
